def userdata( User_id : str ): 
Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.

In [61]:
#importar las librerias
import pandas as pd
import numpy as np

In [62]:
# Leer dataframe de videjuegos solo con las columnas que se necesitan
columns = ['id', 'price']
df_precios = pd.read_csv("steam_games.csv", usecols=columns)
df_precios.head()

,price,id
0,4.99,761140.0
1,Free To Play,643980.0
2,Free to Play,670290.0
3,0.99,767400.0
4,2.99,773570.0


In [63]:
# Verificar que los datatypes estén bien y no haya nulos
df_precios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   price   30758 non-null  object 
 1   id      32133 non-null  float64
dtypes: float64(1), object(1)
memory usage: 502.2+ KB


In [64]:
# Función para corregir el precio, en el que aparecen valores numéricos y en formato string
def fix_price(df):

    errors_list = []
    for i in df['price']:
        try:
            float(i)
        except:
            errors_list.append(i)

    errors = set(errors_list)
    #uniques_not_free = ['Starting at $499.00', 'Starting at $449.00']
    df['price_fixed'] = df['price'].apply(lambda x: 0 if x in errors 
                                                        else 499.0 if x=='Starting at $499.00'
                                                        else 449.0 if x=='Starting at $449.00'
                                                        else x)
    df['price_fixed'] = df['price_fixed'].astype(float)
    return df

In [65]:
# Aplicar la función al dataframe
df_precios = fix_price(df_precios)
# Renombrar la columna de id 
df_precios.rename(columns={'id':'item_id'},inplace=True)
# Eliminar la antigua columna de precio
df_precios.drop(columns='price', inplace=True)
df_precios.head()

,item_id,price_fixed
0,761140.0,4.99
1,643980.0,0.00
2,670290.0,0.00
3,767400.0,0.99
4,773570.0,2.99


In [84]:
df_precios.sample(15)

,item_id,price_fixed
25796,385680.0,7.99
6606,452510.0,29.99
19782,534880.0,9.99
25274,405460.0,6.99
16785,602550.0,8.99
26148,368880.0,4.99
30016,250930.0,7.99
25905,367410.0,0.99
16026,688720.0,14.99
26862,341950.0,9.99


In [44]:
# Verificar nulos
df_precios.isna().sum()

item_id           2
price_fixed    1377
dtype: int64

In [45]:
# Reemplazar nulos en el precio con 0
df_precios.fillna(0, inplace=True)

In [46]:
# Leer el daraframe de user items solo con las columnas que se necesitan
columns = ['steam_id', 'item_id', 'items_count']
user_items = pd.read_csv("game_items.csv", usecols=columns)
user_items.head()
user_items.rename(columns={"steam_id":"user_id"},inplace=True)
user_items["user_id"]=user_items["user_id"].astype(str)

In [47]:
# Hacer un merge (left join) de ambos dataframes on item_id
df_merged = user_items.merge(df_precios, on='item_id', how='left')
df_merged.sample(10)

,user_id,items_count,item_id,price_fixed
2784816,76561198004203748,104,204360,14.99
2487264,76561198045712367,358,7730,4.99
463396,76561198088989575,170,371200,14.99
263750,76561198095150995,77,360,9.99
3883807,76561197977030679,379,207690,9.99
3626321,76561198035287574,232,206190,9.99
1481810,76561198150072019,215,275100,14.99
3496427,76561198022047958,261,550,19.99
3800326,76561198060122780,238,65980,39.99
189368,76561198029850782,77,41070,NaN


In [48]:
# Group by usuario para obtener el total de dinero gastado en videojuegos
user_spent = df_merged.groupby('user_id')['price_fixed'].agg('sum')
user_spent = pd.DataFrame(user_spent)
user_spent.reset_index(inplace=True)
user_spent["user_id"]=user_spent["user_id"].astype(str)

In [49]:
# Leer dataframe de reviews solo con las columnas que se necesitan
columns = ['user_id', 'recommend']
user_revs = pd.read_csv('game_reviews.csv', usecols=columns)
user_revs.sample(5)

,user_id,recommend
9386,giveyouagoodlink,True
338,76561198088944293,False
2294,FORTHMINGUTH,True
40340,tina_1337,True
44898,hektoz,True


In [50]:
# Calcular el porcentaje de recomendación para cada usuario
# Crear un df vacío
user_gb_count_revs = pd.DataFrame()
# Agrupar por user_id y agregar por conteo de recommend para obtener el total de reviews
user_gb_count_revs['count_total'] = user_revs.groupby('user_id')['recommend'].agg('count')
# Contar solo los recommend = True por usuario
user_gb_count_revs['count_true'] = user_revs[user_revs['recommend'] == True].groupby('user_id')['recommend'].agg(count_revs='count')
user_gb_count_revs.reset_index(inplace=True)
# Calcular la proporción de True entre el total
user_gb_count_revs['reco_porcentaje'] = user_gb_count_revs['count_true'] / user_gb_count_revs['count_total'] * 100
user_gb_count_revs.fillna(0,inplace=True)
user_gb_count_revs.sample(5)

,user_id,count_total,count_true,reco_porcentaje
19128,cazaman11,2,2.0,100.0
6726,76561198070917100,1,1.0,100.0
15327,Mastaslim,4,4.0,100.0
699,76561197977316974,1,1.0,100.0
20608,gridalpine,2,2.0,100.0


In [51]:
# Hacer un merge (left join) de los df de reviews y user_spent anteriormente creados
df_merge2 = user_gb_count_revs.merge(user_spent, on='user_id',how='left' )
df_merge2.rename(columns={'price_fixed':'total_spent'},inplace=True)
df_merge2.sample(10)

,user_id,count_total,count_true,reco_porcentaje,total_spent
12670,Brianseb,2,2.0,100.000000,NaN
7164,76561198073094125,1,1.0,100.000000,NaN
8298,76561198080345478,10,10.0,100.000000,574.62
12648,Br0wni3,1,1.0,100.000000,NaN
1550,76561198020449132,1,1.0,100.000000,799.53
24540,themajesticlemon,9,8.0,88.888889,NaN
13569,EnviroReal,9,7.0,77.777778,NaN
13260,DestinyzFate,1,1.0,100.000000,NaN
25353,yourdaemon,1,0.0,0.000000,NaN
7202,76561198073295527,2,1.0,50.000000,272.88


In [52]:
# # Se realiza otro merge con la tabla de user_items para obtener el total de items por usuario
df_merge3 = df_merge2.merge(user_items[['user_id','items_count']].drop_duplicates(),on='user_id',how='left')
# Se eliminan columnas innecesarias
df_merge3 = df_merge3.drop(['count_total','count_true'],axis=1)
# Renombrar columnas
df_merge3.rename(columns={
    'reco_porcentaje':'porcentaje_recomendacion', 'total_spent':'cantidad_gastada', 'items_count':'cantidad_items'},inplace=True)

In [55]:
# Guardar el dataframe final como csv para consumir en la API
df_merge3.to_csv('df-funcion-1.csv',index=False)

In [59]:
# Función de la consulta para obtener el dinero total gastado, cantidad de items 
# y porcentaje de recomendación por usuario
def userdata(user_id : str):
    dfuser = pd.read_csv('df-funcion-1.csv')
    user_data = dfuser[dfuser['user_id'] == user_id]
    
    return user_data.to_json(orient='records')

In [60]:
userdata("MixedMag")	

'[{"user_id":"MixedMag","porcentaje_recomendacion":33.3333333333,"cantidad_gastada":null,"cantidad_items":null}]'